In [1]:
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 1.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
INFO: pip is looking at multiple versions of embedchain to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of embedchain to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
INFO: pip is looking at multiple versions of embedchain[github,youtube] to determin

In [ ]:
from crewai import Agent
from langchain_openai import ChatOpenAI
import os
from dotenv import load_dotenv
from crewai_tools import SerperDevTool


load_dotenv()
os.environ["OPENAI_API_KEY"] = 'YOUR_API_KEY'
os.environ["SERPER_API_KEY"] = 'YOUR_API_KEY'

google_search_tool = SerperDevTool()
llm = ChatOpenAI(model="gpt-4", temperature=0.7)

def create_topic_agent(topic):
    return Agent(
        role="Topic Researcher",
        goal=f"Find high-impact, trending LinkedIn post topics related to {topic}",
        backstory="You are a marketing strategist who studies what's trending on social media, industry blogs, and articles about {topic}.",
        llm=llm,
        tools=[google_search_tool],
    )

writer_agent = Agent(
    role="Content Writer",
    goal="Write multiple variations of a LinkedIn post based on a given topic",
    backstory="You are a creative marketer skilled in writing engaging LinkedIn content in different styles.",
    llm=llm
)

seo_agent = Agent(
    role="SEO Enhancer",
    goal="Improve the LinkedIn post by adding keywords, hashtags, and relevant links",
    backstory="You are a LinkedIn growth hacker specializing in post optimization and discoverability.",
    llm=llm
)


In [3]:
from crewai import Task

def create_tasks(topic_agent, writer_agent, seo_agent, topic):
    topic_task = Task(
        description=f"Generate 3 trending LinkedIn post topics about the importance of {topic}.",
        expected_output="A list of 3 topics.",
        agent=topic_agent
    )

    writer_task = Task(
        description="Create 3 different Linkedin posts in the following styles: 1) quote-style, 2) long-form, and 3) listicle.",
        expected_output="3 content options for the given LinkedIn post topic.",
        agent=writer_agent
    )

    seo_task = Task(
        description="Enhance each post with relevant keywords, hashtags, and an authoritative link.",
        expected_output="SEO-optimized versions of each post.",
        agent=seo_agent
    )

    return topic_task, writer_task, seo_task


In [5]:
from crewai import Crew

if __name__ == "__main__":
    base_topic = input("Enter a broad topic (e.g. AI for mid-market companies): ")

    # Create topic researcher and task
    topic_agent = create_topic_agent(base_topic)
    topic_task, _, _ = create_tasks(topic_agent, writer_agent, seo_agent, base_topic)

    # Run topic generation only
    temp_crew = Crew(
        agents=[topic_agent],
        tasks=[topic_task],
        verbose=True
    )
    topic_result = temp_crew.kickoff()

    print("\nSuggested Topics:")
    topics = [line.strip("- ") for line in topic_result.split("\n") if line.strip()]
    for idx, t in enumerate(topics, 1):
        print(f"{idx}. {t}")

    selected_idx = int(input("\nChoose a topic number to proceed with: ")) - 1
    final_topic = topics[selected_idx]

    # Create agents and full task list based on selected topic
    topic_agent = create_topic_agent(final_topic)
    topic_task, writer_task, seo_task = create_tasks(topic_agent, writer_agent, seo_agent, final_topic)

    crew = Crew(
        agents=[topic_agent, writer_agent, seo_agent],
        tasks=[topic_task, writer_task, seo_task],
        verbose=True
    )

    result = crew.kickoff(inputs={"topic": final_topic})
    print("\nFinal Output:\n")
    print(result)


Enter a broad topic (e.g. AI for mid-market companies): AI for mid-market companies
 [DEBUG]: == Working Agent: Topic Researcher
 [INFO]: == Starting Task: Generate 3 trending LinkedIn post topics about the importance of AI for mid-market companies.
 


Search results: Title: Mid-Market Tech Spending High, and on the Rise with AI ... - LinkedIn
Link: https://www.linkedin.com/pulse/mid-market-tech-spending-high-rise-ai-top-investment-priority-tone-qauec
Snippet: Survey results show the top three technology objectives from the past year – improving cybersecurity, enabling business growth, and optimizing ...
---
Title: Generative AI for Mid-Market Companies - LinkedIn
Link: https://www.linkedin.com/pulse/unlocking-potential-generative-ai-how-mid-market-companies-jones-juklc?trk=organization_guest_main-feed-card_feed-article-content
Snippet: Unlocking the Potential of Generative AI: How Mid-Market Companies Are Thriving with Google Cloud Partners.
---
Title: The 6 Most Powerful AI Marketin

 [DEBUG]: == Working Agent: Topic Researcher
 [INFO]: == Starting Task: Generate 3 trending LinkedIn post topics about the importance of 1. "Unlocking the Potential of Generative AI: Revolutionizing Mid-Market Companies".
 


Search results: Title: Generative AI for Mid-Market Companies - LinkedIn
Link: https://www.linkedin.com/pulse/unlocking-potential-generative-ai-how-mid-market-companies-jones-juklc?trk=organization_guest_main-feed-card_feed-article-content
Snippet: Impact on Mid-Market Companies. Mid-market companies may have the most to gain from generative AI because of their flexibility and agility.
---
Title: Economic potential of generative AI - McKinsey & Company
Link: https://www.mckinsey.com/capabilities/mckinsey-digital/our-insights/the-economic-potential-of-generative-ai-the-next-productivity-frontier
Snippet: Generative AI's impact on productivity could add trillions of dollars in value to the global economy—and the era is just beginning.
---
Title: Unlocking the Potent